# МТС Geohack.112

## Построение таблицы признаков

In [1]:
import pandas as pd
import numpy as np
import collections
import warnings

warnings.filterwarnings('ignore')

df_zones = pd.read_csv('zones.csv', index_col='zone_id')
df_zones.head()

,lat_bl,lon_bl,lat_tr,lon_tr,lat_c,lon_c,is_test,is_target,calls_daily,calls_workday,calls_weekend,calls_wd0,calls_wd1,calls_wd2,calls_wd3,calls_wd4,calls_wd5,calls_wd6
zone_id,,,,,,,,,,,,,,,,,,
0,55.378220,36.999105,55.382744,37.007050,55.380482,37.003077,0,0.0,0.002177,0.001325,0.004213,0.003006,0.000000e+00,6.885802e-04,0.003006,0.000000e+00,0.001374,0.007052
1,55.378252,37.006994,55.382775,37.014938,55.380514,37.010966,0,0.0,0.000996,0.000535,0.002096,0.000765,0.000000e+00,1.176913e-03,0.000765,0.000000e+00,0.001374,0.002818
2,55.378284,37.014883,55.382806,37.022827,55.380545,37.018855,0,0.0,0.001284,0.000724,0.002622,0.001322,0.000000e+00,1.020885e-03,0.001322,0.000000e+00,0.001374,0.003871
3,55.378315,37.022772,55.382837,37.030715,55.380576,37.026744,0,0.0,0.000968,0.000476,0.002143,0.000815,0.000000e+00,7.802124e-04,0.000815,0.000000e+00,0.001374,0.002912
4,55.378345,37.030662,55.382867,37.038603,55.380606,37.034633,0,0.0,0.000842,0.000310,0.002113,0.000783,1.821651e-07,7.286604e-07,0.000782,8.602241e-07,0.001374,0.002851


In [2]:
# Плотность населения Москвы можно приблизительно оценить, зная площадь жилой недвижимости (данные взяты с сайта www.reformagkh.ru)
df_housedata = pd.read_csv('housedata.csv')

df_housedata.loc[df_housedata.AREA_LIVE == 'Незаполнено', 'AREA_LIVE' ] = 0
df_housedata.AREA_LIVE = df_housedata.AREA_LIVE.apply(lambda x: float(x))

df_housedata.loc[df_housedata.AREA_NONLIVE == 'Незаполнено', 'AREA_NONLIVE' ] = 0
df_housedata.AREA_NONLIVE = df_housedata.AREA_NONLIVE.apply(lambda x: float(x))

df_housedata.loc[df_housedata.ROOM_COUNT_NONLIVE == 'Не заполнено', 'ROOM_COUNT_NONLIVE' ] = 0
df_housedata.ROOM_COUNT_NONLIVE = df_housedata.ROOM_COUNT_NONLIVE.apply(lambda x: int(str(x).replace(' ', '')))

df_housedata.loc[df_housedata.ROOM_COUNT_LIVE == 'Не заполнено', 'ROOM_COUNT_LIVE' ] = 0
df_housedata.ROOM_COUNT_LIVE = df_housedata.ROOM_COUNT_LIVE.apply(lambda x: int(str(x).replace(' ', '')))

df_zones_train = df_zones
df_zones_train.sort_values(by='lat_bl')
df_housedata = df_housedata.sort_values(by='LAT')
              
df_zones_train.reset_index(inplace=True)

df_zones_train.head()

,zone_id,lat_bl,lon_bl,lat_tr,lon_tr,lat_c,lon_c,is_test,is_target,calls_daily,calls_workday,calls_weekend,calls_wd0,calls_wd1,calls_wd2,calls_wd3,calls_wd4,calls_wd5,calls_wd6
0,0,55.378220,36.999105,55.382744,37.007050,55.380482,37.003077,0,0.0,0.002177,0.001325,0.004213,0.003006,0.000000e+00,6.885802e-04,0.003006,0.000000e+00,0.001374,0.007052
1,1,55.378252,37.006994,55.382775,37.014938,55.380514,37.010966,0,0.0,0.000996,0.000535,0.002096,0.000765,0.000000e+00,1.176913e-03,0.000765,0.000000e+00,0.001374,0.002818
2,2,55.378284,37.014883,55.382806,37.022827,55.380545,37.018855,0,0.0,0.001284,0.000724,0.002622,0.001322,0.000000e+00,1.020885e-03,0.001322,0.000000e+00,0.001374,0.003871
3,3,55.378315,37.022772,55.382837,37.030715,55.380576,37.026744,0,0.0,0.000968,0.000476,0.002143,0.000815,0.000000e+00,7.802124e-04,0.000815,0.000000e+00,0.001374,0.002912
4,4,55.378345,37.030662,55.382867,37.038603,55.380606,37.034633,0,0.0,0.000842,0.000310,0.002113,0.000783,1.821651e-07,7.286604e-07,0.000782,8.602241e-07,0.001374,0.002851


In [4]:
from tqdm import tqdm

df_zones_train['live_amount'] = 0
df_zones_train['nonlive_amount'] = 0
df_zones_train['room_live'] = 0
df_zones_train['room_nonlive'] = 0

for i, row in tqdm(df_zones_train.iterrows()):
    tmp = df_housedata.query('(LON >= @row.lon_bl) & (LON < @row.lon_tr)')
    tmp = tmp.query('(LAT >= @row.lat_bl) & (LAT < @row.lat_tr)')
    if tmp.shape[0]:
        df_zones_train.loc[i, 'live_amount'] = tmp.AREA_LIVE.sum()
        df_zones_train.loc[i, 'nonlive_amount'] = tmp.AREA_NONLIVE.sum()
        df_zones_train.loc[i, 'room_live'] = tmp.ROOM_COUNT_LIVE.sum()
        df_zones_train.loc[i, 'room_nonlive'] = tmp.ROOM_COUNT_NONLIVE.sum()        

22172it [03:30, 105.17it/s]


In [5]:
# Также будем использовать признаки, которые были построены на основе координат квадратов и данных из OpenStreetMap (из Geohack112_StarterKit.ipynb)
df_features = pd.read_csv('features.csv')

# Добавим дополнительные признаки к уже имеющимся
df_zoness = df_zones_train[['zone_id', 'room_nonlive', 'room_live', 'calls_daily', 'is_test', 'is_target']]
df_features = pd.merge(df_features, df_zoness, on='zone_id')
df_features.head()

,zone_id,distance_to_kremlin,tagged_points_in_0.001,tagged_points_in_0.003,tagged_points_in_0.005,tagged_points_in_0.007,tagged_points_in_0.01,tagged_mean_dist_k_3,tagged_max_dist_k_3,tagged_std_dist_k_3,...,public_transport_std_dist_k_5,public_transport_mean_dist_k_10,public_transport_max_dist_k_10,public_transport_std_dist_k_10,public_transport_min,room_nonlive,room_live,calls_daily,is_test,is_target
0,0,56.852292,0,1,2,2,2,0.006777,0.015403,0.006120,...,0.019011,0.102931,0.119075,0.019160,0.066160,0,0,0.002177,0,0.0
1,1,56.511876,0,0,0,0,2,0.008668,0.010560,0.001347,...,0.018316,0.104594,0.124896,0.019435,0.069525,0,0,0.000996,0,0.0
2,2,56.173822,0,0,0,0,0,0.013769,0.015696,0.002323,...,0.014751,0.106627,0.130933,0.020701,0.073586,0,0,0.001284,0,0.0
3,3,55.838172,0,0,0,0,0,0.020577,0.023549,0.003754,...,0.011964,0.108593,0.133372,0.022350,0.078232,0,0,0.000968,0,0.0
4,4,55.504970,0,0,0,0,0,0.026699,0.030437,0.003545,...,0.010822,0.109584,0.137651,0.023498,0.071765,0,0,0.000842,0,0.0


## Предсказание числа вызовов

#### Обучающая выборка

In [6]:
from sklearn.model_selection import train_test_split

df_zones_tr = df_features.query('is_test == 0 & is_target == 1')

idx_train, idx_valid = train_test_split(df_zones_tr.index, test_size=0.3, random_state=17)

In [7]:
from sklearn.ensemble import RandomForestRegressor
X_train = df_zones_tr.loc[idx_train, :].drop(['calls_daily', 'zone_id', 'is_test', 'is_target'], axis=1)
y_train = df_zones_tr.loc[idx_train, 'calls_daily']

model = RandomForestRegressor(n_estimators=240, n_jobs=-1, random_state=42)

model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=240, n_jobs=-1, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

#### Пример таблицы с предсказаниями (для валидационной выборки)

In [8]:
from scipy.stats import kendalltau

X_valid = df_zones_tr.loc[idx_valid, :].drop(['calls_daily', 'zone_id', 'is_test', 'is_target' ], axis=1)
y_valid = df_zones_tr.loc[idx_valid, 'calls_daily']

y_pred = model.predict(X_valid)

target_columns = ['calls_wd{}'.format(d) for d in range(7)]

df_valid_target = df_zones_train.loc[idx_valid, target_columns]
df_valid_predictions = pd.DataFrame(collections.OrderedDict([
    (column_name, y_pred)
    for column_name in target_columns
]), index=idx_valid)

#### Оценка качества предсказаний на валидационной выборке

In [9]:
df_comparison = pd.DataFrame({
    'target': df_valid_target.unstack(),
    'prediction': df_valid_predictions.unstack(),
})

valid_score = kendalltau(df_comparison['target'], df_comparison['prediction']).correlation
print('Validation score:', valid_score)

Validation score: 0.680034993708


#### Тестовая выборка

In [10]:
idx_test = df_features.query('is_test == 1').index

X_test = df_features.loc[idx_test, :].drop([ 'calls_daily', 'zone_id', 'is_test', 'is_target'], axis=1)
y_pred = model.predict(X_test)

df_test_predictions = pd.DataFrame(collections.OrderedDict([
    (column_name, y_pred)
    for column_name in target_columns
]), index=idx_test)

df_test_predictions.to_csv('sample_submission.csv')
df_test_predictions.head()

,calls_wd0,calls_wd1,calls_wd2,calls_wd3,calls_wd4,calls_wd5,calls_wd6
79,0.016428,0.016428,0.016428,0.016428,0.016428,0.016428,0.016428
80,0.020703,0.020703,0.020703,0.020703,0.020703,0.020703,0.020703
81,0.047453,0.047453,0.047453,0.047453,0.047453,0.047453,0.047453
82,0.055695,0.055695,0.055695,0.055695,0.055695,0.055695,0.055695
83,0.061356,0.061356,0.061356,0.061356,0.061356,0.061356,0.061356
